In [7]:
import pysal as ps
import fw
import time
import numpy as np

In [2]:
N = ps.network.network.Network(ps.examples.get_path('geodanet/streets.shp'))

Here I'll just illustrate the floyd warshall algorithm's usage. 

In [12]:
tfw = time.time()
dfw, pfw = fw.floyd_warshall(N)
tfw = time.time() - tfw
print tfw

3.38626980782


This implementation uses the standard 3-nested for loop structure. It requires a network object, and can take an alternate edge cost dictionary as a keyword. In addition, it can handle directed graphs with a boolean keyword. 

It returns two dicts of dicts, each with outer keys of the origins and inner keys of the destinations. 

`dfw` is the distance dictionary, containing all distances from nodes to nodes. The distance from a node to itself is zero by default. 

`pfw` is the precedence or path dictionary containing all distances from nodes to nodes. This contains the steps to take from start to destination. So, if you have a path $A \rightarrow B \rightarrow E \rightarrow D \rightarrow C$, the list would be $[B, E, D, C]$, as it's four steps from $A$ to $C$.

In [4]:
trd = time.time()
N.node_distance_matrix()
trd = time.time() - trd
print trd

1.16067194939


Floyd-Warshall is slow wrt. repeat dijkstra here. It's got a triple `for` loop through the nodes. But, if `streets.shp` were denser, $V >> N$, it might win out. Do we have a dense network to test that out on?

In [30]:
np.testing.assert_allclose(N.alldistances[0][0], dfw[0].values())

In [37]:
for idx in N.alldistances:
    np.testing.assert_allclose(N.alldistances[idx][0], dfw[idx].values())

But, distances are the same. Predecessor networks are harder to compare due to the behavior I mentioned in [#576](https://github.com/pysal/pysal/issues/576). 

In [72]:
starts_rd = []
nostarts_rd = []

starts_fw = []
nostarts_fw = []
for start in N.alldistances:
    for dest, path in N.alldistances[idx][1].iteritems():
        if start == dest:
            pass
        elif path[-1] != start:
            nostarts_rd.append((start,dest))
        else:
            starts_rd.append((start,dest))
for start in pfw.keys():
    for dest,path in pfw[start].iteritems():
        if start == dest:
            pass
        elif path[0] != start:
            nostarts_fw.append((start,dest))
        else: 
            starts_fw.append((start,dest))

In [73]:
print len(noends_rd), len(ends_rd)
print len(noends_fw), len(ends_fw)

38534 167
52670 0


So, all of the floyd-warshall paths end with the destination node, but some of the repeat dijkstra paths are inconsistent. Sometimes they're the same and both leave out the starting node:

In [69]:
print N.alldistances[0][1][3]
print pfw[0][3][::-1]

[3, 23, 22, 20, 19, 170, 2]
[3, 23, 22, 20, 19, 170, 2]


And sometimes they're different, and dijkstra includes the starting node:

In [65]:
print N.alldistances[229][1][217]
print pfw[229][217][::-1]

[217, 79, 216, 63, 62, 174, 20, 228, 116, 119, 229]
[217, 79, 216, 63, 62, 174, 20, 228, 116, 119]


I can't tell why though. 167 that end in the 